<a href="https://colab.research.google.com/github/fsrt16/Wine-Quality-/blob/master/WineQualityPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dfred = pd.read_csv('winequality-red.csv')
dfwhite = pd.read_csv('winequality-white.csv')
dfred.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [0]:
def pr(df):
  numeric_variables = list(df.select_dtypes(include=['int64','float']).columns.values)
  df[numeric_variables]=df[numeric_variables].apply(lambda x: x.fillna(x.mean()),axis=0)
  return df

In [0]:
dfred = pr(dfred)
dfwhite = pr(dfwhite)

# ML

In [0]:
yred = dfred.quality
ywhite =dfwhite.quality
Xred=dfred.drop('quality',axis=1)
Xwhite=dfwhite.drop('quality',axis=1)

### *For white wine just change the dataset in the below column  ~ Tathagat*

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xred,yred, test_size=0.33, random_state=42)

In [10]:

from sklearn import linear_model
reg = linear_model.Lasso(alpha=0.1)
reg.fit(X_train,y_train)
y_pred = reg.predict(X_test)
from sklearn.metrics import mean_squared_error
from math import sqrt

ms =(mean_squared_error(y_test, y_pred))
ms

0.5300535162863558

In [11]:
from sklearn import linear_model
rerg1 = linear_model.ElasticNet(alpha=0.1)
rerg1.fit(X_train,y_train)
y_pred = rerg1.predict(X_test)
from sklearn.metrics import mean_squared_error
from math import sqrt

ms = (mean_squared_error(y_test, y_pred))
ms

0.5174038262351957

In [12]:
from sklearn import linear_model
rerg1 = linear_model.Ridge(alpha=0.1)
rerg1.fit(X_train,y_train)
y_pred = rerg1.predict(X_test)
from sklearn.metrics import mean_squared_error
from math import sqrt

ms = (mean_squared_error(y_test, y_pred))
ms

0.4244355904504343

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
#rng = np.random.RandomState(1)
regr_1 = DecisionTreeRegressor(max_depth=5)
regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),n_estimators=300)

regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)

# Predict
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)
rms1 = (mean_squared_error(y_test, y_1))
print(rms1)
rms2 =(mean_squared_error(y_test, y_2))
print(rms2)

0.4785533686555068
0.4117238574757246


In [14]:
from sklearn.ensemble import GradientBoostingRegressor
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0, loss='ls').fit(X_train, y_train)
print(mean_squared_error(y_test, est.predict(X_test)) )

0.4396248349928679


In [38]:
from xgboost.sklearn import XGBRegressor
#model = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.95,
                    # colsample_bytree=0.95, reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
model = XGBRegressor(max_depth=9,min_child_weight = 1)
model.fit(X_train, y_train)
#print(model)
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = (mean_squared_error(y_test, y_pred))
print(rms)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


0.36951833828075104


In [0]:
#from sklearn.model_selection import cross_validation, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
param_grid = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
logreg_cv = GridSearchCV(model, param_grid, cv = 5) 
logreg_cv.fit(X_train, y_train,eval_metric='rmse')


In [35]:
print("Tuned XGB Parameters: {}".format(logreg_cv.best_params_))  
print("Best score is {}".format(logreg_cv.best_score_))

Tuned XGB Parameters: {'max_depth': 9, 'min_child_weight': 3}
Best score is 0.44108929387128276


In [0]:
from catboost import CatBoostRegressor
#model = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=8, min_child_weight=6, gamma=0.1, subsample=0.95,
                    # colsample_bytree=0.95, reg_alpha=2, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
modelc = CatBoostRegressor(depth=8,iterations=30,learning_rate=0.1)
modelc.fit(X_train, y_train)
print(modelc)
y_pred = modelc.predict(X_test)
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = (mean_squared_error(y_test, y_pred))
rms

In [30]:
rms

0.6080083725958267

In [0]:

from sklearn.model_selection import GridSearchCV
  
model = CatBoostRegressor()
parameters = {'depth'         : [6,8,10],
                  'learning_rate' : [0.01, 0.05, 0.1],
                  'iterations'    : [30, 50, 100]
                 }
grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
grid.fit(X_train, y_train)

In [22]:
print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
print("\n The best score across ALL searched params:\n",
          grid.best_score_)
print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)


 The best estimator across ALL searched params:

 The best score across ALL searched params:
 4.3902808808634255

 The best parameters across ALL searched params:
 {'depth': 10, 'iterations': 30, 'learning_rate': 0.01}


### ***Hence best achieved result is of XGBOSST with hyper param 9 depth and 1 child weight . With a 0.36 Root mean square error***